<a href="https://colab.research.google.com/github/arielba2002/Deep-Picker-Project/blob/model%2Ffinal-model%2Binference/model/Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Model and Training Scalers

Import modules

In [1]:
import json
import pickle
from tensorflow.keras.models import load_model

Download files

In [2]:
# Download the model file
!gdown --id 1o4mj7otvgTp_-36_DFlLcpPyPnSKfPBX -O enhanced_attention_model.h5

# Download the x_scaler
!gdown --id 1OCoCaOEKwVTxbm8pamFfyLu8wNbkRg2W -O x_scaler.pkl

# Download the y_scaler
!gdown --id 1h6TGOQWvQY3FZ7uNAmmjn1k4yUeV_Tkj -O y_scaler.pkl

# Download json
!gdown --id 1tD_yAa_R3eb-ssKWOu-oqxlRH0lVvQJ_ -O previous_year_data.json


# load previous year teams json
data_path = '/content/previous_year_data.json'

with open(data_path, 'r') as json_data:
    data = json.load(json_data)

print("Sample player data:", json.dumps(data['LAL_2020']["players"][0], indent=4))

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1o4mj7otvgTp_-36_DFlLcpPyPnSKfPBX
To: /content/enhanced_attention_model.h5
100% 14.5M/14.5M [00:00<00:00, 77.3MB/s]
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1OCoCaOEKwVTxbm8pamFfyLu8wNbkRg2W
To: /content/x_scaler.pkl
100% 1.68k/1.68k [00:00<00:00, 7.10MB/s]
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
F

consts

In [3]:
player_list = [
    {
        "id": 18215,
        "playerName": "Anthony Gill",
        "position": "PF",
        "age": 31,
        "games": 50,
        "gamesStarted": 3,
        "minutesPg": 466.0,
        "fieldGoals": 75,
        "fieldAttempts": 160,
        "fieldPercent": 0.469,
        "threeFg": 10,
        "threeAttempts": 41,
        "threePercent": 0.244,
        "twoFg": 65,
        "twoAttempts": 119,
        "twoPercent": 0.546,
        "effectFgPercent": 0.5,
        "ft": 29,
        "ftAttempts": 36,
        "ftPercent": 0.806,
        "offensiveRb": 37,
        "defensiveRb": 57,
        "totalRb": 94,
        "assists": 34,
        "steals": 13,
        "blocks": 11,
        "turnovers": 23,
        "personalFouls": 48,
        "points": 189,
        "team": "WAS",
        "season": 2024,
        "playerId": "gillan01",
        "image": "https://cdn.nba.com/headshots/nba/latest/1040x760/1630264.png"
    },
    {
        "id": 18120,
        "playerName": "Bilal Coulibaly",
        "position": "SF",
        "age": 19,
        "games": 63,
        "gamesStarted": 15,
        "minutesPg": 1715.0,
        "fieldGoals": 188,
        "fieldAttempts": 432,
        "fieldPercent": 0.435,
        "threeFg": 64,
        "threeAttempts": 185,
        "threePercent": 0.346,
        "twoFg": 124,
        "twoAttempts": 247,
        "twoPercent": 0.502,
        "effectFgPercent": 0.509,
        "ft": 92,
        "ftAttempts": 131,
        "ftPercent": 0.702,
        "offensiveRb": 56,
        "defensiveRb": 200,
        "totalRb": 256,
        "assists": 110,
        "steals": 57,
        "blocks": 48,
        "turnovers": 87,
        "personalFouls": 137,
        "points": 532,
        "team": "WAS",
        "season": 2024,
        "playerId": "coulibi01",
        "image": "https://cdn.nba.com/headshots/nba/latest/1040x760/1641731.png"
    },
    {
        "id": 18334,
        "playerName": "Corey Kispert",
        "position": "SF",
        "age": 24,
        "games": 80,
        "gamesStarted": 22,
        "minutesPg": 2067.0,
        "fieldGoals": 386,
        "fieldAttempts": 794,
        "fieldPercent": 0.486,
        "threeFg": 184,
        "threeAttempts": 480,
        "threePercent": 0.383,
        "twoFg": 202,
        "twoAttempts": 314,
        "twoPercent": 0.643,
        "effectFgPercent": 0.602,
        "ft": 114,
        "ftAttempts": 157,
        "ftPercent": 0.726,
        "offensiveRb": 30,
        "defensiveRb": 195,
        "totalRb": 225,
        "assists": 156,
        "steals": 41,
        "blocks": 16,
        "turnovers": 98,
        "personalFouls": 141,
        "points": 1070,
        "team": "WAS",
        "season": 2024,
        "playerId": "kispeco01",
        "image": "https://cdn.nba.com/headshots/nba/latest/1040x760/1630557.png"
    },
    {
        "id": 18195,
        "playerName": "Daniel Gafford",
        "position": "C",
        "age": 25,
        "games": 45,
        "gamesStarted": 45,
        "minutesPg": 1192.0,
        "fieldGoals": 203,
        "fieldAttempts": 294,
        "fieldPercent": 0.69,
        "threeFg": 0,
        "threeAttempts": 0,
        "threePercent": 0,
        "twoFg": 203,
        "twoAttempts": 294,
        "twoPercent": 0.69,
        "effectFgPercent": 0.69,
        "ft": 84,
        "ftAttempts": 119,
        "ftPercent": 0.706,
        "offensiveRb": 131,
        "defensiveRb": 230,
        "totalRb": 361,
        "assists": 69,
        "steals": 44,
        "blocks": 97,
        "turnovers": 46,
        "personalFouls": 147,
        "points": 490,
        "team": "WAS",
        "season": 2024,
        "playerId": "gaffoda01",
        "image": "https://cdn.nba.com/headshots/nba/latest/1040x760/1629655.png"
    },
    {
        "id": 18198,
        "playerName": "Danilo Gallinari",
        "position": "PF",
        "age": 35,
        "games": 26,
        "gamesStarted": 0,
        "minutesPg": 385.0,
        "fieldGoals": 57,
        "fieldAttempts": 131,
        "fieldPercent": 0.435,
        "threeFg": 20,
        "threeAttempts": 64,
        "threePercent": 0.313,
        "twoFg": 37,
        "twoAttempts": 67,
        "twoPercent": 0.552,
        "effectFgPercent": 0.511,
        "ft": 47,
        "ftAttempts": 56,
        "ftPercent": 0.839,
        "offensiveRb": 12,
        "defensiveRb": 64,
        "totalRb": 76,
        "assists": 31,
        "steals": 6,
        "blocks": 3,
        "turnovers": 12,
        "personalFouls": 26,
        "points": 181,
        "team": "WAS",
        "season": 2024,
        "playerId": "gallida01",
        "image": "https://cdn.nba.com/headshots/nba/latest/1040x760/201568.png"
    },
    {
        "id": 18705,
        "playerName": "Delon Wright",
        "position": "PG",
        "age": 31,
        "games": 33,
        "gamesStarted": 0,
        "minutesPg": 456.0,
        "fieldGoals": 48,
        "fieldAttempts": 122,
        "fieldPercent": 0.393,
        "threeFg": 14,
        "threeAttempts": 38,
        "threePercent": 0.368,
        "twoFg": 34,
        "twoAttempts": 84,
        "twoPercent": 0.405,
        "effectFgPercent": 0.451,
        "ft": 24,
        "ftAttempts": 29,
        "ftPercent": 0.828,
        "offensiveRb": 13,
        "defensiveRb": 46,
        "totalRb": 59,
        "assists": 82,
        "steals": 35,
        "blocks": 8,
        "turnovers": 11,
        "personalFouls": 22,
        "points": 134,
        "team": "WAS",
        "season": 2024,
        "playerId": "wrighde01",
        "image": "https://cdn.nba.com/headshots/nba/latest/1040x760/1626153.png"
    },
    {
        "id": 18000,
        "playerName": "Deni Avdija",
        "position": "SF",
        "age": 23,
        "games": 75,
        "gamesStarted": 75,
        "minutesPg": 2257.0,
        "fieldGoals": 406,
        "fieldAttempts": 803,
        "fieldPercent": 0.506,
        "threeFg": 88,
        "threeAttempts": 235,
        "threePercent": 0.374,
        "twoFg": 318,
        "twoAttempts": 568,
        "twoPercent": 0.56,
        "effectFgPercent": 0.56,
        "ft": 202,
        "ftAttempts": 273,
        "ftPercent": 0.74,
        "offensiveRb": 84,
        "defensiveRb": 456,
        "totalRb": 540,
        "assists": 286,
        "steals": 60,
        "blocks": 36,
        "turnovers": 160,
        "personalFouls": 185,
        "points": 1102,
        "team": "WAS",
        "season": 2024,
        "playerId": "avdijde01",
        "image": "https://cdn.nba.com/headshots/nba/latest/1040x760/1630166.png"
    },
    {
        "id": 18485,
        "playerName": "Eugene Omoruyi",
        "position": "SF",
        "age": 26,
        "games": 43,
        "gamesStarted": 0,
        "minutesPg": 393.0,
        "fieldGoals": 81,
        "fieldAttempts": 167,
        "fieldPercent": 0.485,
        "threeFg": 13,
        "threeAttempts": 46,
        "threePercent": 0.283,
        "twoFg": 68,
        "twoAttempts": 121,
        "twoPercent": 0.562,
        "effectFgPercent": 0.524,
        "ft": 32,
        "ftAttempts": 49,
        "ftPercent": 0.653,
        "offensiveRb": 35,
        "defensiveRb": 50,
        "totalRb": 85,
        "assists": 35,
        "steals": 27,
        "blocks": 5,
        "turnovers": 21,
        "personalFouls": 62,
        "points": 207,
        "team": "WAS",
        "season": 2024,
        "playerId": "omorueu01",
        "image": "https://cdn.nba.com/headshots/nba/latest/1040x760/1630647.png"
    }]

unwanted_stats = ["id", "playerName", "team", "season", "playerId", "gamesStarted", "image"]
per_minute_stats = [
    "points", "assists", "steals", "blocks", "turnovers", "personalFouls",
    "offensiveRb", "defensiveRb", "totalRb", "fieldGoals", "fieldAttempts",
    "threeFg", "threeAttempts", "twoFg", "twoAttempts", "ft", "ftAttempts"
]

Load scalers and model files

In [4]:
# Load scalers from disk
import tensorflow as tf

with open('x_scaler.pkl', 'rb') as f:
    x_scaler = pickle.load(f)

with open('y_scaler.pkl', 'rb') as f:
    y_scaler = pickle.load(f)

In [5]:
# Load the model from the .h5 file
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Lambda # Import Lambda layer


import tensorflow as tf
from tensorflow.keras.models import load_model

def mean_pool(t):
    return tf.reduce_mean(t, axis=1)

def max_pool(t):
    return tf.reduce_max(t, axis=1)

custom_objects = {
    'mean_pool': mean_pool,
    'max_pool': max_pool
}

model = load_model('enhanced_attention_model.h5', custom_objects=custom_objects)

# # Define custom objects for Lambda layers
# def mean_pool_fn(t):
#     return tf.reduce_mean(t, axis=1)

# def max_pool_fn(t):
#     return tf.reduce_max(t, axis=1)

# # Create a dictionary of custom objects
# custom_objects = {
#     'Lambda': Lambda, # Include the Keras Lambda layer itself if needed
#     'mean_pool_fn': mean_pool_fn,
#     'max_pool_fn': max_pool_fn,
#     # Based on the model code, the lambda functions were used directly.
#     # We need to map the names Keras assigned to these lambdas during saving
#     # to our defined functions. Keras often uses names like 'lambda'.
#     'lambda': mean_pool_fn, # Assuming the first lambda was for mean
#     'lambda_1': max_pool_fn, # Assuming the second lambda was for max
#     "tf": tf
# }


# # Load the model with custom objects
# try:
#     model = load_model('enhanced_attention_model.h5', custom_objects=custom_objects)
# except Exception as e:
#     print(f"Error loading model with custom objects: {e}")
#     # Fallback might not work, but keeping it just in case
#     model = load_model('enhanced_attention_model.h5')

In [6]:
def remove_unwanted_stats(df, stats_to_remove):
    """Removes unwanted statistic columns from the DataFrame."""
    return df.drop(columns=stats_to_remove, errors="ignore")

def normalize_columns(df, cols, divisor_col):
    """Normalizes specified columns by dividing them by a divisor column."""
    df = df.copy()
    for col in cols:
        if col in df.columns and divisor_col in df.columns:
            df[col] = df.apply(lambda row: row[col] / row[divisor_col] if row[divisor_col] != 0 else 0, axis=1)
    return df

def one_hot_encode_positions(df, position_col="position"):
    """
    Converts the player position column into one-hot encoded columns,
    ensuring consistent output for the 5 standard basketball positions.

    :param df: Player statistics DataFrame.
    :param position_col: Name of the column containing player positions.
    :return: DataFrame with one-hot encoded position columns.
    """
    if position_col in df.columns:
        standard_positions = ["PG", "SG", "SF", "PF", "C"]

        # Initialize zero columns
        for pos in standard_positions:
            df[f"{position_col}_{pos}"] = 0

        # Iterate through rows to set appropriate columns to 1
        for idx, value in df[position_col].dropna().items():
            # Normalize and split multi-position strings
            positions = [p.strip().upper() for p in value.replace("/", "-").split("-")]
            for pos in positions:
                col_name = f"{position_col}_{pos}"
                if col_name in df.columns:
                    df.at[idx, col_name] = 1

        # Drop the original column
        df = df.drop(columns=[position_col])

    return df

In [7]:
import pandas as pd
import numpy as np
def preprocess_player_list(player_list, stats_to_remove=unwanted_stats, per_minute_stats=per_minute_stats,
                                minutes_col="minutesPg", position_col="position"):
    """End-to-end pipeline to preprocess player list entry."""
    df = pd.DataFrame(player_list)
    df = remove_unwanted_stats(df, stats_to_remove)
    df = normalize_columns(df, per_minute_stats, minutes_col)
    df = one_hot_encode_positions(df, position_col)
    X = df.to_numpy()
    X = x_scaler.transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
    X = X[np.newaxis, :, :] # Add batch dimension

    return X

In [8]:
def print_prediction(prediction):
  performance_labels = [
    "Points", "Assists", "Rebounds", "Blocks", "Steals",
    "Win %", "Conf Rank", "FGM", "FGA", "3PM", "3PA",
    "FTM", "FTA", "OREB", "DREB", "Fouls", "Turnovers"
  ]
  print("\n🏆 SAMPLE TEAM PREDICTIONS:")
  print("-" * 80)
  print("Metric          Prediction")
  print("-" * 45)

  for j, label in enumerate(performance_labels):  # Show first 8 metrics
      predicted = prediction[0, j]
      print(f"{label:12s}: {predicted:10.1f}")

# Inference

In [9]:
import tensorflow as tf # Keep import here as well as a safeguard
team_key = "LAL_2023"
data_copy = data.copy()

prediction = y_scaler.inverse_transform(model.predict(preprocess_player_list(player_list)))
print_prediction(prediction)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 800ms/step

🏆 SAMPLE TEAM PREDICTIONS:
--------------------------------------------------------------------------------
Metric          Prediction
---------------------------------------------
Points      :      108.5
Assists     :       24.2
Rebounds    :       43.0
Blocks      :        4.5
Steals      :        7.7
Win %       :        0.4
Conf Rank   :       12.6
FGM         :       40.7
FGA         :       88.3
3PM         :       11.2
3PA         :       33.1
FTM         :       16.2
FTA         :       21.2
OREB        :       10.1
DREB        :       33.5
Fouls       :       20.2
Turnovers   :       14.7
